In [1]:
import torch
from torchmetrics import Accuracy, F1Score
from transformers import XLNetForQuestionAnsweringSimple, AutoTokenizer
from dataset import SquadDataset

In [2]:
# Define save and load path
xlnet_dir = "model_weights/XLNet/"

load_path = xlnet_dir + "06-04-2023-17-30" # add date_time as name

In [3]:
# Define hyperparameters
batch_size = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased", padding_side="right")
model = XLNetForQuestionAnsweringSimple.from_pretrained(load_path).to(device)

In [5]:
# Load dataset and dataloaders
squad_dataset = SquadDataset(tokenizer, max_length=384, batch_size=batch_size, eval=True)
eval_dataloader = squad_dataset.eval_dataloader

eval_dataset = squad_dataset.eval_dataset

Found cached dataset squad (C:/Users/Dell/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [6]:
def inference(idx, eval_dataloader, eval_dataset, model, tokenizer):
    
    raw_sample = eval_dataset[idx]
    input_sample = eval_dataloader.dataset[idx]
    
    input_ids = input_sample["input_ids"].unsqueeze(0).to(device)
    attention_mask = input_sample["attention_mask"].unsqueeze(0).to(device)
    start_positions = input_sample["start_positions"].unsqueeze(0).to(device)
    end_positions = input_sample["end_positions"].unsqueeze(0).to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    
    start_span = outputs.start_logits.cpu().argmax()
    end_span = outputs.end_logits.cpu().argmax()
    
    answer_tokens = input_ids[0, start_span:end_span+1]
    predicted_answer = tokenizer.decode(answer_tokens)
    
    question = raw_sample["question"]
    context = raw_sample["context"]
    gt_answer = raw_sample["answers"]
    
    print("Context:\n" + context + "\n\nQuestion:\n" + question + "\n\nGround Truth answer:\n", gt_answer, 
          "\n\nPredicted answer:\n" + predicted_answer)

In [7]:
inference(4903, eval_dataloader, eval_dataset, model, tokenizer)

Context:
Internet2 is a not-for-profit United States computer networking consortium led by members from the research and education communities, industry, and government. The Internet2 community, in partnership with Qwest, built the first Internet2 Network, called Abilene, in 1998 and was a prime investor in the National LambdaRail (NLR) project. In 2006, Internet2 announced a partnership with Level 3 Communications to launch a brand new nationwide network, boosting its capacity from 10 Gbit/s to 100 Gbit/s. In October, 2007, Internet2 officially retired Abilene and now refers to its new, higher capacity network as the Internet2 Network.

Question:
Who did internet2 partner with 

Ground Truth answer:
 {'text': ['a partnership with Level 3 Communications to launch a brand new nationwide network', 'Level 3 Communications', 'Qwest'], 'answer_start': [368, 387, 206]} 

Predicted answer:
Qwest


In [8]:
inference(10000, eval_dataloader, eval_dataset, model, tokenizer)

Context:
With Istanbul as its capital and control of lands around the Mediterranean basin, the Ottoman Empire was at the center of interactions between the Eastern and Western worlds for six centuries. Following a long period of military setbacks against European powers, the Ottoman Empire gradually declined into the late nineteenth century. The empire allied with Germany in the early 20th century, with the imperial ambition of recovering its lost territories, but it dissolved in the aftermath of World War I, leading to the emergence of the new state of Turkey in the Ottoman Anatolian heartland, as well as the creation of modern Balkan and Middle Eastern states, thus ending Turkish colonial ambitions.

Question:
Which country today is a remnant of the Ottoman empire?

Ground Truth answer:
 {'text': ['Turkey', 'Turkey', 'Turkey', 'Turkey', 'Turkey'], 'answer_start': [551, 551, 551, 551, 551]} 

Predicted answer:
Germany


In [9]:
inference(125, eval_dataloader, eval_dataset, model, tokenizer)

Context:
The league eventually narrowed the bids to three sites: New Orleans' Mercedes-Benz Superdome, Miami's Sun Life Stadium, and the San Francisco Bay Area's Levi's Stadium.

Question:
What three stadiums did the NFL decide between for the game?

Ground Truth answer:
 {'text': ["New Orleans' Mercedes-Benz Superdome, Miami's Sun Life Stadium, and the San Francisco Bay Area's Levi's Stadium", "New Orleans' Mercedes-Benz Superdome, Miami's Sun Life Stadium, and the San Francisco Bay Area's Levi's Stadium.", "New Orleans' Mercedes-Benz Superdome, Miami's Sun Life Stadium, and the San Francisco Bay Area's Levi's Stadium."], 'answer_start': [56, 56, 56]} 

Predicted answer:
New Orleans' Mercedes-Benz Superdome


In [11]:
import random

idx = random.randint(0,len(eval_dataset))

inference(idx, eval_dataloader, eval_dataset, model, tokenizer)

Context:
Exceptional examples of the bourgeois architecture of the later periods were not restored by the communist authorities after the war (like mentioned Kronenberg Palace and Insurance Company Rosja building) or they were rebuilt in socialist realism style (like Warsaw Philharmony edifice originally inspired by Palais Garnier in Paris). Despite that the Warsaw University of Technology building (1899–1902) is the most interesting of the late 19th-century architecture. Some 19th-century buildings in the Praga district (the Vistula’s right bank) have been restored although many have been poorly maintained. Warsaw’s municipal government authorities have decided to rebuild the Saxon Palace and the Brühl Palace, the most distinctive buildings in prewar Warsaw.

Question:
What style was the Warsaw Philharmony edifice built in?

Ground Truth answer:
 {'text': ['socialist realism', 'socialist realism', 'socialist realism'], 'answer_start': [229, 229, 229]} 

Predicted answer:
socialist rea